In [ ]:
import pandas as pd
import geopandas as gp
import numpy as np
import rasterstats
import os
import fiona
import rasterio
import rasterio.mask

In [ ]:
output_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Neighborhood_Files"
model_name = "sp"

In [ ]:
# set building height
default_height = 20

In [ ]:
# set project crs
crs = 2926

In [ ]:
# set extent boundary
extent_path = f"/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Boundary/{model_name}_extent.shp"
extent = gp.read_file(extent_path)

## TODO: create a shapefile for grasshopper
## add one blank row

In [ ]:
# floodplain extent for clipping
floodplain_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Boundary/floodplain.shp"
floodplain = gp.read_file(floodplain_path)

In [ ]:
# clip parcels to floodplain
parcel_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Raw/Building_Parcels.shp"
parcels = gp.read_file(parcel_path)
parcels.to_crs(crs, inplace = True)

parcels['centroid_geom'] = parcels.centroid
parcels.set_geometry('centroid_geom',inplace = True)
parcels_clip = gp.overlay(parcels, floodplain, how = "intersection")
parcels_clip = gp.overlay(parcels_clip, extent, how = "intersection")

In [ ]:
parcels_merge = pd.merge(parcels_clip, parcels[['OBJECTID','geometry']], left_on ='OBJECTID', right_on = 'OBJECTID', how = 'left')
parcels_merge.set_geometry('geometry_y', inplace = True)

In [ ]:
parcels_merge.drop(['geometry_x'], axis=1, inplace=True)
parcels_merge.to_file(os.path.join(output_path, f"{model_name}_neighborhood_parcels.shp"))

ROADS PROCESSING

In [ ]:
# roads processing
roads_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Projected/seattle_streets_projected.shp"
roads = gp.read_file(roads_path)
roads_clip = gp.overlay(roads, extent, how='intersection')
roads_clip.to_crs(crs, inplace = True)
## TODO: dissolve roads
roads_clip.to_file(os.path.join(output_path, f"{model_name}_neighborhood_streets.shp"))

BUILDINGS PROCESSING

In [ ]:
buildings_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/buildings_projected_trim.shp"
buildings = gp.read_file(buildings_path)

In [ ]:
buildings['centroid_geom'] = buildings.centroid
buildings.set_geometry('centroid_geom',inplace = True)
buildings_clip = gp.overlay(buildings, floodplain, how = "intersection")
buildings_clip = gp.overlay(buildings_clip, extent, how = "intersection")

In [ ]:
buildings_merge = pd.merge(buildings_clip, buildings[['OBJECTID','geometry']], left_on ='OBJECTID', right_on = 'OBJECTID', how = 'left')
buildings_merge.set_geometry('geometry_y', inplace = True)

In [ ]:
# fill 0 and NA values to default_height
buildings_merge['BP99_APEX'].fillna(default_height, inplace = True)
buildings_merge['BP99_APEX'] = np.where(buildings_merge['BP99_APEX']==0,default_height, buildings_merge['BP99_APEX'])
buildings_merge['BP99_APEX'] = buildings_merge['BP99_APEX'].astype(int)

In [ ]:
buildings_merge.to_crs(crs, inplace = True)
buildings_merge['AREA'] = buildings_merge.area
buildings_merge[['OBJECTID', 'BP99_TYPE', 'BP99_APEX','AREA','geometry_y']].to_file(os.path.join(output_path,f"{model_name}_neighborhood_buildings.shp"))

In [ ]:
geom = buildings_merge.envelope
buildings_merge.set_geometry(geom, inplace = True)
buildings_merge[['OBJECTID', 'BP99_TYPE', 'BP99_APEX','geometry_y']].to_file(os.path.join(output_path, f"{model_name}_neighborhood_buildings_bb.shp"))

In [65]:
parcel_building_join = gp.sjoin(buildings_merge, parcels_merge, how = 'left')

### TODO: set geometry for single family homes and multi family as bounding, everything else stays the same
parcel_building_join[['LAND_USE_1','geometry_y']].to_file(os.path.join(output_path, f"{model_name}_neighborhood_buildings_bb_test.shp"))

In [ ]:
parcel_building_single = parcel_building_join.sort_values(by=['PIN','AREA']).drop_duplicates(subset=['PIN'], keep='last')
parcel_building_single[['OBJECTID_left', 'BP99_TYPE', 'BP99_APEX','geometry_y']].to_file(os.path.join(output_path, f"{model_name}_neighborhood_buildings_bb_trim.shp"))

CONTOUR PROCESSING

In [ ]:
# clip catherine's topo
contours_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/DV_GIS Topo Data/06_GIS Data/02_Clipping Layers/01_Duwamish Valley/DV King Country Contours_clipped.shp"
contours = gp.read_file(contours_path)
contours.to_crs(crs, inplace = True)
contours_clip = gp.overlay(contours, extent, how = 'intersection')

In [ ]:
contours_clip_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Neighborhood_Files/neighborhood_contours_clip.shp"
contours_clip = gp.read_file(contours_clip_path)

In [ ]:
# if elevation > 0, take every contour 10
contours_below_SL = contours_clip[contours_clip['ELEVATION']<=20]
contours_above_SL = contours_clip[contours_clip['IDX_20FT']==20]
contours_above_SL = contours_above_SL[contours_above_SL['ELEVATION']>20]
contours_trimmed = pd.concat([contours_below_SL,contours_above_SL])

In [ ]:
contours_trimmed.to_file(os.path.join(output_path, "neighborhood_contours_clip_test.shp"))

TOPO PROCESSING
- merging bathymetry data with topography dem

In [ ]:
# mask topo_rast by extent
topo_rast = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Neighborhood_Files/topo.tif"

with fiona.open(extent_path, "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

with rasterio.open(topo_rast) as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta

In [ ]:
topo_outpath = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Neighborhood_Files/neighborhood_topo.asc"

# convert to ascii file
out_meta.update({"driver": "AAIGrid",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

with rasterio.open(topo_outpath, "w", **out_meta) as dest:
    dest.write(out_image)

EXTRA PROCESSING FOR ASSIGNING ABSOLUTE BUILDING HEIGHTS FROM TERRAIN

In [ ]:
## used topo to raster tool with 5ft contours and 5 ft pixel resolution
# there may be better ways to get DEM / elevation values as raster
topo_rast = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/min_bath_topo.tif"
zonal_stats = rasterstats.zonal_stats(buildings_merge, topo_rast, stats="count min mean max median std")
stats_df = pd.DataFrame.from_dict(zonal_stats).reset_index()
buildings_merge.reset_index(inplace = True)
merged_heights = pd.merge(buildings_merge, stats_df, how = 'left', left_on = 'index', right_on = 'index')
merged_heights['height_upd'] = np.where((merged_heights['BP99_APEX'].isna()==False) | (merged_heights['mean'].isna()==False), default_height, (merged_heights['BP99_APEX']-merged_heights['mean']))
merged_heights.set_geometry('geometry_y')
merged_heights[['OBJECTID','BP99_APEX','mean','min','max','median','std','height_upd','geometry_y']].to_file("/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/building_heights_upd.shp")